In [2]:
!pip install nodejs geopandas contextily

  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 994 kB 25.0 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 6.5 MB 55.9 MB/s            
     |████████████████████████████████| 2.0 MB 52.7 MB/s            
     |████████████████████████████████| 16.6 MB 46.8 MB/s            
     |████████████████████████████████| 19.2 MB 64.8 MB/s            
     |████████████████████████████████| 118 kB 73.8 MB/s            
  Created wheel for nodejs: filename=nodejs-0.1.1-py3-none-any.whl size=3492 sha256=87c325b830bd271ad5415ea3d4cec236a1504a226839d112cace422895284003
  Stored in directory: /home/ec2-user/.cache/pip/wheels/e0/2f/be/4ee779e16832b901c856255265149d584e14e519d32e7875c5
  Created wheel for optional-django: filename=optional_django-0.1.0-py3-none-any.whl size=9980 sha256=4494f232c637ab02da4433378a9e7b1493964a09fcd2800070cd399bd5044741
  Stored in directory: /home/ec2-user/.cache/pip/wheels/d9/41/fa/ec371

In [12]:
import os
from helper_functions.shared_functions import *
import boto3
import geopandas as gpd
import pandas as pd

# os.environ['EGIS_DB_HOST'] = "hv-ti-egis-rds-pg-egdb.c4vzypepnkx3.us-east-1.rds.amazonaws.com"  #TI DB
os.environ['EGIS_DB_HOST'] = "hv-ti-egis-rds-pg-egdb.c4vzypepnkx3.us-east-1.rds.amazonaws.com"  #Dev DB
os.environ['EGIS_DB_DATABASE'] = "hydrovis"
os.environ['EGIS_DB_USERNAME'] = "hydrovis"
os.environ['EGIS_DB_PASSWORD'] = "hardwork123donehere"

db_type = "egis"
db_engine = get_db_engine(db_type)

s3 = boto3.client('s3')

# Define bucket and parent directories.
bucket = "hydrovis-ti-deployment-us-east-1"
parent_directory = "qc_fim_data"
local_download_parent_directory = f'brad_data/qc_fim_data'

#file_handles = ['flow_based_catfim.csv', 'flow_based_catfim_sites.csv',
#               'stage_based_catfim.csv', 'stage_based_catfim_sites.csv']

#file_handles = ['fim_performance_points.csv', 'fim_performance_polys.csv']

#file_handles = ['fim_performance_catchments.csv']

#file_handles = ['stage_based_catfim_sites.csv']

file_handles = ['flow_based_catfim.csv']
#file_handles = ['stage_based_catfim_sites.csv']

for file_handle in file_handles:
    # Define path to file to download and its local download path, the download.
    filename = f"{parent_directory}/{file_handle}"
    print(filename)
    local_download_path = os.path.join(local_download_parent_directory, f'{file_handle}')
    print(f"--> Downloading {bucket}/{filename} to {local_download_path}")
    #s3.download_file(bucket, filename, local_download_path)
    
    #  -- Open file and reformat -- #
    print("Reading file...")
    df = pd.read_csv(local_download_path)
    print("File read.")
    # Rename headers.
    if file_handle == 'fim_performance_points.csv':
        df = df.rename(columns={'Unnamed: 0': 'oid', 'geometry': 'geom'})
    else:
        df = df.rename(columns={'Unnamed: 0': 'oid', 'geometry': 'geom', 'huc':'huc8'})
        
    if file_handle == 'stage_based_catfim.csv':
        pass
        # Simplify and clean up status messages before loading into DB
    if file_handle == 'fim_performance_catchments.csv':
        #df = df.drop('valid', axis=1)
        print(df.shape)
 #       df.insert(0, 'oid', range(0, len(df)))
    # Convert all field names to lowercase (needed for ArcGIS Pro).
    df.columns= df.columns.str.lower()

    # Remove sites that are in derived.ahps_restricted_sites
    if "catfim" in file_handle:
        restricted_sites_df = get_db_values("derived.ahps_restricted_sites", "*")
        restricted_dict = restricted_sites_df.to_dict('records')
        
        # Change 'mapped' to 'no' if sites are present in restricted_sites_df
        for site in restricted_dict:
            nws_lid = site['nws_lid'].lower()
            if "sites" in file_handle:
                print(True)
                print(nws_lid)
                df.loc[df.ahps_lid==nws_lid, 'mapped'] = 'no'
                df.loc[df.ahps_lid==nws_lid, 'status'] = site['restricted_reason']
                print(df.loc[df.ahps_lid==nws_lid]['status'])
            else:
                df.loc[df.ahps_lid==nws_lid, 'viz'] = 'no'
    
        # Enforce data types on df before loading in DB (TODO: need to create special cases for each layer).
    if file_handle == 'fim_performance_points.csv':
        df = df.astype({'huc': 'str'})
    else:
        df = df.astype({'huc8': 'str'})
    df = df.fillna(0)
    try:
        df = df.astype({'feature_id': 'int'})
        df = df.astype({'feature_id': 'str'})
    except KeyError:  # If there is no feature_id field
        pass
    try:
        df = df.astype({'nwm_seg': 'int'})
        df = df.astype({'nwm_seg': 'str'})
    except KeyError:  # If there is no nwm_seg field
        pass
    try:
        df = df.astype({'usgs_gage': 'int'})
        df = df.astype({'usgs_gage': 'str'})
    except KeyError:  # If there is no usgs_gage field
        pass
        
    # zfill HUC8 field.
    if file_handle == 'fim_performance_points.csv':
        df['huc'] = df['huc'].apply(lambda x: x.zfill(8))
    else:
        df['huc8'] = df['huc8'].apply(lambda x: x.zfill(8))
    
    if file_handle in ['flow_based_catfim_sites.csv', 'stage_based_catfim_sites.csv']:
        df = df.astype({'nws_data_rfc_forecast_point': 'str'})
        df = df.astype({'nws_data_rfc_defined_fcst_point': 'str'})
        df = df.astype({'nws_data_riverpoint': 'str'})
        
    # Upload df to database.
    stripped_layer_name = file_handle.replace(".csv", "")
    table_name = "reference." + stripped_layer_name
    print("Loading data into DB...")
    
    print("Dataframe shape")
    print(df.shape[0])
    
    # Chunk load data into DB
    if file_handle in ['fim_performance_catchments.csv', 'stage_based_catfim.csv', 'flow_based_catfim.csv']:
        print("Chunk loading...")
        # Create list of df chunks
        n = 100000  #chunk row size
        list_df = [df[i:i+n] for i in range(0,df.shape[0],n)]
        print("list_df len")
        print(len(list_df))
        
        # Load the first chunk into the DB as a new table
        first_chunk_df = list_df[0]
        print(first_chunk_df.shape[0])
        load_df_into_db(table_name, db_engine, first_chunk_df, epsg=3857, drop_first=True)
        
        # Load remaining chunks into newly created table
        for remaining_chunk in list_df[1:]:
            print(remaining_chunk.shape[0])
            load_df_into_db(table_name, db_engine, remaining_chunk, epsg=3857, drop_first=False)
        db_engine.execute(f'CREATE INDEX ON reference.{stripped_layer_name} USING GIST (geom);')
        
    else:
        load_df_into_db(table_name, db_engine, df, epsg=3857, drop_first=True)
        db_engine.execute(f'CREATE INDEX ON reference.{stripped_layer_name} USING GIST (geom);')
        
    if file_handle == 'fim_performance_catchments.csv':
        print("Making valid...")
        make_valid_sql = f"""

        ALTER TABLE reference.fim_performance_catchments
          ADD COLUMN geom_invalid geometry
          DEFAULT NULL;

        UPDATE reference.fim_performance_catchments
          SET geom = ST_MakeValid(geom),
              geom_invalid = geom
          WHERE NOT ST_IsValid(geom);
          """

        db_engine.execute(make_valid_sql)

qc_fim_data/flow_based_catfim.csv
--> Downloading hydrovis-ti-deployment-us-east-1/qc_fim_data/flow_based_catfim.csv to brad_data/qc_fim_data/flow_based_catfim.csv
Reading file...
File read.
Retrieving values for *
Loading data into DB...
Dataframe shape
12854
Chunk loading...
list_df len
1
12854
Dropping reference.flow_based_catfim if it exists
Getting sql to create table
CREATE TABLE reference.flow_based_catfim (
"oid" INTEGER,
  "ahps_lid" TEXT,
  "magnitude" TEXT,
  "version" TEXT,
  "huc8" TEXT,
  "interval_stage" DOUBLE PRECISION,
  "name" TEXT,
  "wfo" TEXT,
  "rfc" TEXT,
  "state" TEXT,
  "county" TEXT,
  "q" DOUBLE PRECISION,
  "q_uni" TEXT,
  "q_src" TEXT,
  "stage" DOUBLE PRECISION,
  "stage_uni" TEXT,
  "s_src" TEXT,
  "wrds_time" TEXT,
  "nrldb_time" TEXT,
  "nwis_time" TEXT,
  "lat" DOUBLE PRECISION,
  "lon" DOUBLE PRECISION,
  "viz" TEXT,
  "geom" GEOMETRY(Geometry,3857)
)
Creating reference.flow_based_catfim
Adding data to reference.flow_based_catfim


In [3]:
print(df[:6])

   oid ahps_lid usgs_gage  nwm_seg      huc8 mapped  \
0    0    00brd         0  8211323  18060005    yes   
1    1    00rdr         0  8200819  18060005    yes   
2    2    brdc1  11150500  8211251  18060005    yes   
3    3    chlc1  11152300  8200795  18060005     no   
4    4    kcyc1  11151300  8205487  18060005    yes   
5    5    lwdc1  11149900  8210533  18060005    yes   

                                              status  \
0                  record is missing calculated flow   
1                     all calculated flows available   
2                  record is missing calculated flow   
3                           missing calculated flows   
4  minor is missing calculated flow, major is mis...   
5  minor is missing calculated flow, major is mis...   

                                           geom  
0  POINT (-13467864.90530122 4290726.437706408)  
1  POINT (-13550921.61426519 4376566.834692953)  
2  POINT (-13455052.32257835 4291027.211735288)  
3  POINT (-13530815.7

In [3]:
from helper_functions.shared_functions import *
from brad_data.variables import service_desc_dict

os.environ['VIZ_DB_USERNAME'] = "viz_proc_admin_rw_user"
os.environ['VIZ_DB_PASSWORD'] = "QLQW31ZxL0CuE78Rm1lvjYTWg"

for service_name in service_desc_dict:
    print("Updating " + service_name + "...")
    folder = service_desc_dict[service_name]['folder']
    configuration = service_desc_dict[service_name]['configuration']
    summary = service_desc_dict[service_name]['summary']
    description = service_desc_dict[service_name]['description']
    tags = service_desc_dict[service_name]['tags']
    credits = service_desc_dict[service_name]['credits']

    sql = f"""
    INSERT INTO admin.services(
    service, folder, configuration, summary, description, tags, credits, run, feature_service, server, fim_service)
    VALUES ('{service_name}', '{folder}', '{configuration}', '{summary}', '{description}', '{tags}', '{credits}', false, false, 'server', false)
    """
    run_sql_in_db(sql, db_type="viz")
    


Updating fim_performance...


IntegrityError: (psycopg2.IntegrityError) duplicate key value violates unique constraint "services_pkey"
DETAIL:  Key (service)=(fim_performance) already exists.

[SQL: 
    INSERT INTO admin.services(
    service, folder, configuration, summary, description, tags, credits, run, feature_service, server, fim_service)
    VALUES ('fim_performance', 'eval', 'reference', 'Height Above Nearest Drainage (HAND) Flood Inundation Mapping (FIM) Performance at HUC8s and AHPS locations.', 'Depicts FIM prediction skill metrics at HUC8s and AHPS locations. Metrics computed through static comparison of benchmark maps to HAND maps at specific magnitudes, e.g. 100yr, 500yr, Action, Minor, Moderate, Major. Metrics reported as Critical Success Index (CSI), True Positive Rate (TPR), False Alarm Ratio (FAR), and Probability Not Detected (PND). For a description of how metrics are computed, see https://github.com/NOAA-OWP/inundation-mapping/wiki/Evaluating-HAND-Performance. Benchmark sources include FEMA Ba